# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
import pandas as pd
from scipy import stats
import numpy as np

In [75]:
#Your code here

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [65]:
exp = df[df.group == 'experiment']
contr = df[df.group == 'control']

In [66]:
exp.id.value_counts()

841724    2
339175    2
447462    2
687449    2
781172    2
         ..
231882    1
309593    1
502214    1
352709    1
735865    1
Name: id, Length: 2996, dtype: int64

In [67]:
view = df[df.action == 'view']
click = df[df.action == 'click']

In [68]:
#checking if anyone clicked but didn't view
click['id'].isin(view['id']).sum() == len(click.id)

True

In [ ]:
"Everyone that clicked also viewed"

In [69]:
#checking for overlap between the two groups ie 
#if there are ids in the experimental group that are in the control group

np.isin(exp.id.unique(),contr.id.unique()).sum()

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [70]:
#Your code here
print(exp.shape,contr.shape)

(3924, 4) (4264, 4)


In [ ]:
"""
A welch ttest is used as the two groups have different sample sizes
h_0: There is no difference between experimental and control homepages
h_1: The experimental homepage is more effective than the control homepage
"""

In [76]:
exp['click'] = 0
exp.loc[exp.action == 'click','click'] = 1
contr['click'] = 0
contr.loc[contr.action == 'click','click'] = 1

In [77]:
contr.click.value_counts()

0    3332
1     932
Name: click, dtype: int64

In [78]:
stats.ttest_ind(contr.click,exp.click,equal_var=False,alternative='less')

Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.026743886922199395)

In [ ]:
"""
The p value is smaller than the alpha therefore we can reject the null hypothesis
"""

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [79]:
#Your code here
click_rate = contr.click.sum()/len(contr.click)
exp_clicks = click_rate * len(exp.click)
exp_clicks

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [80]:
click_rate

0.21857410881801126

In [81]:
#Your code here
n = len(exp.click)
p = click_rate
v = n*p*(1-p)
std = np.sqrt(v)

In [82]:
z=(exp.click.sum()-exp_clicks)/std

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [83]:
#Your code here
1-stats.norm.cdf(z)

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: At the end of the day the p-value in both tests is lower than the alpha of 0.05 and we can therefore reject the null hypothesis. The experimental homepage is indeed more effective than the control homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.